In [12]:
import os
import fnmatch

# Define the directory where the .tif files are located
directory = 'H:\My Drive\GRACE_downscaling (1)'

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file ends with "(1).tif"
    if fnmatch.fnmatch(filename, '* (1).tif'):
        file_path = os.path.join(directory, filename)
        try:
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")


Deleted: H:\My Drive\GRACE_downscaling (1)\vegetation_indices_2002_11 (1).tif
Deleted: H:\My Drive\GRACE_downscaling (1)\snow_cover_2002_10 (1).tif
Deleted: H:\My Drive\GRACE_downscaling (1)\vegetation_indices_2002_10 (1).tif
Deleted: H:\My Drive\GRACE_downscaling (1)\lst_2002_09 (1).tif
Deleted: H:\My Drive\GRACE_downscaling (1)\snow_cover_2002_11 (1).tif
Deleted: H:\My Drive\GRACE_downscaling (1)\vegetation_indices_2002_09 (1).tif
Deleted: H:\My Drive\GRACE_downscaling (1)\lst_2002_10 (1).tif
Deleted: H:\My Drive\GRACE_downscaling (1)\lst_2002_11 (1).tif
Deleted: H:\My Drive\GRACE_downscaling (1)\snow_cover_2002_12 (1).tif
Deleted: H:\My Drive\GRACE_downscaling (1)\vegetation_indices_2002_08 (1).tif
Deleted: H:\My Drive\GRACE_downscaling (1)\snow_cover_2002_09 (1).tif
Deleted: H:\My Drive\GRACE_downscaling (1)\vegetation_indices_2002_07 (1).tif
Deleted: H:\My Drive\GRACE_downscaling (1)\lst_2002_08 (1).tif
Deleted: H:\My Drive\GRACE_downscaling (1)\lst_2003_01 (1).tif
Deleted: H:\My 

In [3]:
import ee
ee.Initialize(project = 'ee-074bce151saurav')

# Study area
roi = ee.Geometry.Rectangle([80.05, 26.35, 88.20, 30.45])
start_date = '2002-01-01'
end_date = '2013-12-31'

def export_data(image, description, scale=1000):
    task = ee.batch.Export.image.toDrive(
        image=image.toFloat(),  # Convert all bands to same type
        folder = 'Downscaling',
        description=description,
        scale=scale,
        region=roi,
        crs='EPSG:4326',
        maxPixels=1e13
    )
    task.start()
    return task

# GLDAS collection with more variables
gldas = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H") \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .select([
        'SoilMoi0_10cm_inst',    # Soil moisture
        'SoilMoi10_40cm_inst',
        'SoilMoi40_100cm_inst',
        'Rainf_f_tavg',          # Rainfall rate
        'Tair_f_inst',           # Air temperature
        'Qair_f_inst',           # Specific humidity
        'SWE_inst',              # Snow water equivalent
        'Evap_tavg',             # Total evapotranspiration
        'Qs_acc',                # Surface runoff
        'Qsb_acc'                # Subsurface runoff
    ])

# FLDAS collection
fldas = ee.ImageCollection("NASA/FLDAS/NOAH01/C/GL/M/V001") \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .select([
        'SoilMoi00_10cm_tavg',   # Soil moisture
        'SoilMoi10_40cm_tavg',
        'SoilMoi40_100cm_tavg',
        'Rainf_f_tavg',            # Rainfall
        'Tair_f_tavg',           # Air temperature
        'Qair_f_tavg',           # Specific humidity
        'Evap_tavg',             # Evapotranspiration
        'Qs_tavg',               # Surface runoff
        'Qsb_tavg'               # Subsurface runoff
    ])

# CHIRPS Precipitation
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
    .filterBounds(roi) \
    .filterDate(start_date, end_date)

# ERA5-Land monthly data
era5_land = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY") \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .select([
        'total_precipitation',    
        'surface_pressure',       
        'temperature_2m',         
        'soil_temperature_level_1',  
        'volumetric_soil_water_layer_1',  
        'surface_runoff'          
    ])

def maskL7clouds(image):
    qa = image.select('QA_PIXEL')
    cloud = qa.bitwiseAnd(1 << 3).eq(0)
    shadow = qa.bitwiseAnd(1 << 4).eq(0)
    return image.updateMask(cloud.And(shadow))

def calculate_indices(image):
    ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
    ndwi = image.normalizedDifference(['SR_B4', 'SR_B5']).rename('NDWI')
    return image.addBands([ndvi, ndwi])

landsat7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
    .filterBounds(roi) \
    .filterDate(start_date, end_date) \
    .map(maskL7clouds) \
    .map(calculate_indices) \
    .select(['NDVI', 'NDWI'])

def process_monthly_data():
    tasks = []
    
    for year in range(2002, 2014):
        for month in range(1, 13):
            start_date = ee.Date.fromYMD(year, month, 1)
            end_date = start_date.advance(1, 'month')
            
            # GLDAS monthly (cast to Float32)
            gldas_monthly = gldas \
                .filterDate(start_date, end_date) \
                .mean() \
                .toFloat() \
                .rename([
                    'gldas_soil_0_10', 'gldas_soil_10_40', 'gldas_soil_40_100',
                    'gldas_rainfall', 'gldas_temp', 'gldas_humidity',
                    'gldas_swe', 'gldas_et', 'gldas_runoff_surf', 'gldas_runoff_sub'
                ])
            
            # FLDAS monthly (cast to Float32)
            fldas_monthly = fldas \
                .filterDate(start_date, end_date) \
                .mean() \
                .toFloat() \
                .rename([
                    'fldas_soil_0_10', 'fldas_soil_10_40', 'fldas_soil_40_100',
                    'fldas_rainfall', 'fldas_temp', 'fldas_humidity',
                    'fldas_et', 'fldas_runoff_surf', 'fldas_runoff_sub'
                ])
            
            # CHIRPS monthly (cast to Float32)
            chirps_monthly = chirps \
                .filterDate(start_date, end_date) \
                .mean() \
                .toFloat() \
                .rename('chirps_precip')
            
            # ERA5 monthly (cast to Float32)
            era5_monthly = era5_land \
                .filterDate(start_date, end_date) \
                .mean() \
                .toFloat() \
                .rename([
                    'era5_precip', 'era5_pressure', 'era5_temp',
                    'era5_soil_temp', 'era5_soil_moisture', 'era5_runoff'
                ])
            
            # Landsat indices monthly (cast to Float32)
            indices_monthly = landsat7 \
                .filterDate(start_date, end_date) \
                .mean() \
                .toFloat() \
                .rename(['ndvi', 'ndwi'])
            
            # Combine all bands
            monthly_composite = ee.Image.cat([
                gldas_monthly,
                fldas_monthly,
                chirps_monthly,
                era5_monthly,
                indices_monthly
            ])
            
            # Print band names for verification
            print(f"\nBands for {year}-{month}:", 
                  monthly_composite.bandNames().getInfo())
            
            task = export_data(
                image=monthly_composite,
                description=f'monthly_data_{year}_{month:02d}',
                scale=1000
            )
            
            tasks.append(task)
            
    return tasks

def export_static_data():
    # DEM and derivatives
    dem = ee.Image('USGS/SRTMGL1_003').clip(roi).toFloat()
    terrain = ee.Terrain.products(dem).toFloat()
    
    # Soil properties (select specific bands and rename)
    soil_bulk_density = ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02") \
        .select('b0') \
        .clip(roi) \
        .toFloat() \
        .rename('bulk_density')
    
    soil_texture = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02") \
        .select('b0') \
        .clip(roi) \
        .toFloat() \
        .rename('soil_texture')
    
    # Surface water
    water = ee.Image('JRC/GSW1_4/GlobalSurfaceWater') \
        .select('occurrence') \
        .clip(roi) \
        .toFloat() \
        .rename('water_occurrence')
    
    # Combine all static data with proper names
    static_composite = ee.Image.cat([
        dem.rename('elevation'),
        terrain.select('slope').rename('slope'),
        terrain.select('aspect').rename('aspect'),
        soil_bulk_density,
        soil_texture,
        water
    ])
    
    print("\nStatic data bands:", static_composite.bandNames().getInfo())
    
    return export_data(
        image=static_composite,
        description='static_data',
        scale=250
    )

# Run exports
try:
    print("Starting monthly data export...")
    monthly_tasks = process_monthly_data()
    
    print("\nStarting static data export...")
    #static_task = export_static_data()
    
    print("\nAll export tasks have been started!")
    print("Check your Google Earth Engine Tasks tab to monitor progress.")
    print("Files will be downloaded to your Google Drive.")
    
except Exception as e:
    print(f"An error occurred: {str(e)}")

Starting monthly data export...

Bands for 2002-1: ['gldas_soil_0_10', 'gldas_soil_10_40', 'gldas_soil_40_100', 'gldas_rainfall', 'gldas_temp', 'gldas_humidity', 'gldas_swe', 'gldas_et', 'gldas_runoff_surf', 'gldas_runoff_sub', 'fldas_soil_0_10', 'fldas_soil_10_40', 'fldas_soil_40_100', 'fldas_rainfall', 'fldas_temp', 'fldas_humidity', 'fldas_et', 'fldas_runoff_surf', 'fldas_runoff_sub', 'chirps_precip', 'era5_precip', 'era5_pressure', 'era5_temp', 'era5_soil_temp', 'era5_soil_moisture', 'era5_runoff', 'ndvi', 'ndwi']

Bands for 2002-2: ['gldas_soil_0_10', 'gldas_soil_10_40', 'gldas_soil_40_100', 'gldas_rainfall', 'gldas_temp', 'gldas_humidity', 'gldas_swe', 'gldas_et', 'gldas_runoff_surf', 'gldas_runoff_sub', 'fldas_soil_0_10', 'fldas_soil_10_40', 'fldas_soil_40_100', 'fldas_rainfall', 'fldas_temp', 'fldas_humidity', 'fldas_et', 'fldas_runoff_surf', 'fldas_runoff_sub', 'chirps_precip', 'era5_precip', 'era5_pressure', 'era5_temp', 'era5_soil_temp', 'era5_soil_moisture', 'era5_runoff', 

In [2]:
import os

# Specify the path to the parent folder
parent_folder = r"C:\Users\J01013381\OneDrive - Jackson State University\Research Projects\2025\ORISE\GW_Downscale\Approach2"

# Walk through the directory
for root, dirs, files in os.walk(parent_folder):
    print(f"Directory: {root}")
    for dir_name in dirs:
        print(f"  Subfolder: {dir_name}")
    for file_name in files:
        print(f"  File: {file_name}")


Directory: C:\Users\J01013381\OneDrive - Jackson State University\Research Projects\2025\ORISE\GW_Downscale\Approach2
  Subfolder: Code
  Subfolder: Data
  Subfolder: logs
  Subfolder: models
Directory: C:\Users\J01013381\OneDrive - Jackson State University\Research Projects\2025\ORISE\GW_Downscale\Approach2\Code
  Subfolder: grace_downscaling
  Subfolder: grace_downscaling.egg-info
  Subfolder: scripts
  Subfolder: tests
  File: Extract Grace.ipynb
  File: README.md.txt
  File: requirements.txt
  File: setup.py
  File: try.ipynb
Directory: C:\Users\J01013381\OneDrive - Jackson State University\Research Projects\2025\ORISE\GW_Downscale\Approach2\Code\grace_downscaling
  Subfolder: data
  Subfolder: models
  Subfolder: utils
  Subfolder: __pycache__
  File: __init__.py
Directory: C:\Users\J01013381\OneDrive - Jackson State University\Research Projects\2025\ORISE\GW_Downscale\Approach2\Code\grace_downscaling\data
  Subfolder: __pycache__
  File: data_diagnostics.py
  File: data_generator

In [ ]:
import ee
ee.Initialize(project='ee-074bce151saurav')

# Study area 
roi = ee.Geometry.Rectangle([80.05, 26.35, 88.20, 30.45])
start_date = '2002-01-01'
end_date = '2013-12-31'

def process_grace_data():
    """
    Process GRACE LWE data for the region of interest
    """
    grace = ee.ImageCollection("NASA/GRACE/MASS_GRIDS_V03/MASCON_CRI") \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .select('lwe_thickness')
    
    tasks = []
    
    for year in range(2002, 2014):
        for month in range(1, 13):
            start_month = ee.Date.fromYMD(year, month, 1)
            end_month = start_month.advance(1, 'month')
            
            # Get monthly GRACE LWE data
            grace_monthly = grace \
                .filterDate(start_month, end_month) \
                .mean() \
                .toFloat() \
                .rename('grace_lwe')
            
            # Export the data
            task = ee.batch.Export.image.toDrive(
                image=grace_monthly,
                description=f'grace_lwe_{year}_{month:02d}',
                folder='GRACE_Data_another',
                scale=55660,
                region=roi,
                crs='EPSG:4326',
                maxPixels=1e13
            )
            
            task.start()
            tasks.append(task)
            
            print(f"Started export for GRACE LWE data: {year}-{month:02d}")
    
    return tasks

# Run the GRACE export
try:
    print("Starting GRACE LWE data export...")
    grace_tasks = process_grace_data()
    print("\nAll GRACE export tasks have been started!")
    print("Check your Google Earth Engine Tasks tab to monitor progress.")
    print("Files will be downloaded to your Google Drive in the GRACE_Data folder.")
    
except Exception as e:
    print(f"An error occurred: {str(e)}")

: 